**Requirements:**
1. numpy
2. pandas
3. matplotlib
4. pillow
5. plotly
6. nbformat
7. ipywidgets
8. modelbit
9. torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
10. transformers
11. scikit-learn

# **0. Important bit: Using ModelBit for the model deployment**

In [ ]:
import modelbit
modelbit.login()

<module 'modelbit' from 'd:\\FoodSnap\\.venv\\Lib\\site-packages\\modelbit\\__init__.py'>

In [ ]:
def Deploying_Model_to_modelbit(model, model_name="FoodSnap_V50e"):
    modelbit.add_model(model_name, model)
    print(f"Model '{model_name}' has been deployed to Modelbit.")

# **1. Getting our Dataset.**

In [ ]:
import os
import plotly.io as pio

In [ ]:
from PIL import Image
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
data_dir = 'Food Snap Dataset'
categories = os.listdir(data_dir)
categories.sort()

In [ ]:
images = []
labels = []

for category in categories:
    category_dir = os.path.join(data_dir, category)
    image_path = os.path.join(category_dir, os.listdir(category_dir)[0])
    images.append(image_path)
    labels.append(category)

In [ ]:
num_cols = 4
num_rows = (len(categories) // num_cols) + 1
fig = make_subplots(rows=num_rows, cols=num_cols, subplot_titles=labels)

for i, img_path in enumerate(images):
    img = Image.open(img_path).resize((224, 224))
    row = (i // num_cols) + 1
    col = (i % num_cols) + 1
    fig.add_trace(
        go.Image(z=img),
        row=row, col=col
    )

fig.update_layout(
    title=f"List of Labels in the Dataset({len(categories)} labels)",
    showlegend=False,
    width=num_cols * 300,
    height=num_rows * 300
)
fig.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)
fig.show()

## **2. Data Preprocessing Steps**



1. importing the necessary libraries
2. creating two transform functions for train and valid sets
3. splitting the dataset into train, validation, and test in 7:1:2 ratio.
4. applying the transform functions on train and validation sets and creating dataloaders for train, validation, and test sets.
5. finally displaying a batch of training set after the processing is applied.



In [ ]:
import torch
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
dataset = ImageFolder(root=data_dir)
class_distribution = []
for idx, (class_name, count) in enumerate(sorted(dataset.class_to_idx.items(), key=lambda x: x[1])):
    num_images = len([y for y in dataset.targets if y == idx])
    class_distribution.append((class_name, num_images))

print(f"Number of classes: {len(dataset.classes)}")
class_df = pd.DataFrame(class_distribution, columns=["Class Name", "Number of Images"])
class_df

Number of classes: 80


,Class Name,Number of Images
0,adhirasam,50
1,aloo_gobi,50
2,aloo_matar,50
3,aloo_methi,50
4,aloo_shimla_mirch,50
...,...,...
75,shrikhand,50
76,sohan_halwa,50
77,sohan_papdi,50
78,sutar_feni,50


In [ ]:
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size

train_data, val_data, test_data = random_split(
    dataset,
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

train_data.dataset.transform = train_transform
val_data.dataset.transform = val_test_transform
test_data.dataset.transform = val_test_transform

num_workers = min(2, os.cpu_count() or 1)
batch_size = 32

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)

print(f"Total images: {total_size}")
print(f"Training images: {len(train_data)}")
print(f"Validation images: {len(val_data)}")
print(f"Test images: {len(test_data)}")

Total images: 4000
Training images: 2800
Validation images: 400
Test images: 800


In [ ]:
def imshow(img):
    img = img / 2 + 0.5
    img = img.clamp(0, 1)
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(train_loader)
images, labels = next(dataiter)
imshow(torchvision.utils.make_grid(images))

# **3. training dr2**

In [ ]:
import torch
import torch.nn as nn

from transformers import AutoImageProcessor, SwinForImageClassification
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from torchvision import transforms
from tqdm import tqdm

In [ ]:
processor = AutoImageProcessor.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
model = SwinForImageClassification.from_pretrained("microsoft/swin-tiny-patch4-window7-224")

model.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(model.classifier.in_features, 80)
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=2, factor=0.5, verbose=True)

preprocessor_config.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/113M [00:00<?, ?B/s]

d:\FoodSnap\.venv\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning:

The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.



In [ ]:
num_epochs = 50
patience = 15
best_val_accuracy = 0.0
best_train_accuracy = 0.0
epochs_no_improve = 0
early_stop = False

In [ ]:
for epoch in range(num_epochs):
    if early_stop:
        print("Early stopping triggered.")
        break

    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct_predictions += (preds == labels).sum().item()
        total_predictions += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct_predictions / total_predictions
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

    model.eval()
    val_loss = 0.0
    correct_val_predictions = 0
    total_val_predictions = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct_val_predictions += (preds == labels).sum().item()
            total_val_predictions += labels.size(0)

    val_loss /= len(val_loader)
    val_accuracy = correct_val_predictions / total_val_predictions
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}\n")

    scheduler.step(val_accuracy)

    if val_accuracy > best_val_accuracy or (val_accuracy == best_val_accuracy and epoch_accuracy > best_train_accuracy):
        best_val_accuracy = val_accuracy
        best_train_accuracy = epoch_accuracy
        epochs_no_improve = 0
        torch.save(model.state_dict(), "best_model.pth")
        print("Validation accuracy improved or training accuracy improved with same validation accuracy; model saved.\n")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print(f"No improvement in validation accuracy or training accuracy for {patience} epochs. Early stopping.\n")
            early_stop = True

Training Epoch 1/50: 100%|██████████| 88/88 [00:41<00:00,  2.12it/s]


Epoch [1/50], Loss: 3.4777, Accuracy: 0.1886
Validation Loss: 2.1940, Validation Accuracy: 0.4150

Validation accuracy improved or training accuracy improved with same validation accuracy; model saved.



Training Epoch 2/50: 100%|██████████| 88/88 [00:40<00:00,  2.19it/s]


Epoch [2/50], Loss: 1.5286, Accuracy: 0.5943
Validation Loss: 1.3332, Validation Accuracy: 0.6325

Validation accuracy improved or training accuracy improved with same validation accuracy; model saved.



Training Epoch 3/50: 100%|██████████| 88/88 [00:39<00:00,  2.22it/s]


...

Epoch [42/50], Loss: 0.0132, Accuracy: 0.9957
Validation Loss: 1.1050, Validation Accuracy: 0.7500



Training Epoch 43/50: 100%|██████████| 88/88 [00:56<00:00,  1.57it/s]


Epoch [43/50], Loss: 0.0160, Accuracy: 0.9954
Validation Loss: 1.1046, Validation Accuracy: 0.7500



Training Epoch 44/50: 100%|██████████| 88/88 [00:39<00:00,  2.20it/s]


Epoch [44/50], Loss: 0.0140, Accuracy: 0.9961
Validation Loss: 1.1045, Validation Accuracy: 0.7500

No improvement in validation accuracy or training accuracy for 15 epochs. Early stopping.

Early stopping triggered.


In [ ]:
import matplotlib.pyplot as plt

epochs = list(range(1, 45))
train_accuracy = [
    0.1886, 0.5943, 0.7893, 0.8914, 0.9311, 0.9754, 0.9879, 0.9814, 0.9857, 0.9839,
    0.9825, 0.9896, 0.9921, 0.9932, 0.9936, 0.9943, 0.9954, 0.9950, 0.9957, 0.9954,
    0.9943, 0.9929, 0.9939, 0.9939, 0.9939, 0.9950, 0.9939, 0.9950, 0.9964, 0.9957,
    0.9946, 0.9932, 0.9943, 0.9939, 0.9954, 0.9950, 0.9939, 0.9964, 0.9950, 0.9939,
    0.9957, 0.9957, 0.9954, 0.9961
]
val_accuracy = [
    0.4150, 0.6325, 0.6800, 0.7075, 0.7150, 0.7275, 0.7275, 0.7325, 0.7275, 0.7200,
    0.7125, 0.7150, 0.7500, 0.7275, 0.7300, 0.7225, 0.7500, 0.7400, 0.7400, 0.7325,
    0.7425, 0.7500, 0.7500, 0.7500, 0.7550, 0.7500, 0.7525, 0.7525, 0.7550, 0.7450,
    0.7550, 0.7525, 0.7500, 0.7500, 0.7500, 0.7500, 0.7500, 0.7500, 0.7500, 0.7500,
    0.7500, 0.7500, 0.7500, 0.7500
]


plt.figure(figsize=(10, 6))
plt.plot(epochs, train_accuracy, label='Training Accuracy', marker='o')
plt.plot(epochs, val_accuracy, label='Validation Accuracy', marker='x')

plt.axvline(x=25, color='orange', linestyle='--', label='Fluctuation starts')
plt.axvline(x=29, color='green', linestyle='--', label='Best Model Saved at Epoch 29')
plt.axvline(x=33, color='red', linestyle='--', label='True Plateau starts')

plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy Over Epochs')
plt.legend()
plt.grid()

plt.tight_layout()
plt.savefig('Epoch History.png')
plt.show()

In [ ]:
import time
from sklearn.metrics import classification_report, confusion_matrix

best_model = SwinForImageClassification.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
best_model.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(best_model.classifier.in_features, 80)
)
state_dict = torch.load("best_model.pth", weights_only=True)
best_model.load_state_dict(state_dict)

Deploying_Model_to_modelbit(best_model,"FoodSnap_V50")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_model.to(device)
best_model.eval()

test_preds = []
test_labels = []
start_time = time.time()

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = best_model(images).logits
        _, preds = torch.max(outputs, 1)

        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

end_time = time.time()

test_report = classification_report(test_labels, test_preds, target_names=categories, zero_division=0)
print("Classification Report:")
print(test_report)

Encrypting 'FoodSnap_V50': 100%|██████████| 106M/106M [00:00<00:00, 280MB/s] 
Uploading 'FoodSnap_V50': 100%|██████████| 97.5M/97.5M [01:08<00:00, 1.49MB/s]
Updating registry: 100%|██████████| 1/1 [00:02<00:00]


Model 'FoodSnap_V50' has been deployed to Modelbit.
Classification Report:
                              precision    recall  f1-score   support

                   adhirasam       0.42      0.56      0.48         9
                   aloo_gobi       0.73      0.92      0.81        12
                  aloo_matar       0.69      0.69      0.69        13
                  aloo_methi       1.00      1.00      1.00         9
           aloo_shimla_mirch       0.58      0.78      0.67         9
                  aloo_tikki       0.88      0.88      0.88         8
                      anarsa       0.78      0.70      0.74        10
                     ariselu       0.64      0.58      0.61        12
                bandar_laddu       1.00      0.90      0.95        10
                     basundi       0.56      0.50      0.53        10
                     bhatura       0.92      0.92      0.92        13
               bhindi_masala       0.93      1.00      0.96        13
              

In [ ]:
correct_predictions = np.sum(np.array(test_preds) == np.array(test_labels))
total_predictions = len(test_labels)

test_accuracy = correct_predictions / total_predictions
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 75.38%


In [ ]:
test_conf_matrix = confusion_matrix(test_labels, test_preds)
test_conf_matrix_normalized = test_conf_matrix.astype('float') / test_conf_matrix.sum(axis=1)[:, np.newaxis]

precision = np.diag(test_conf_matrix) / np.sum(test_conf_matrix, axis=0)
recall = np.diag(test_conf_matrix) / np.sum(test_conf_matrix, axis=1)
f1 = 2 * (precision * recall) / (precision + recall)

fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Normalized Confusion Matrix", "Per-class Performance Metrics"),
    vertical_spacing=0.15,
    row_heights=[0.6, 0.4]
)

fig.add_trace(
    go.Heatmap(
        z=test_conf_matrix_normalized,
        x=categories,
        y=categories,
        colorscale='Blues',
        colorbar=dict(
            title="Normalized Value",
            y=0.8,
            len=0.5
        ),
        hoverongaps=False,
        hovertemplate="True: %{y}<br>Predicted: %{x}<br>Value: %{z:.2f}<extra></extra>",
        zmin=0,
        zmax=1
    ),
    row=1, col=1
)

fig.add_trace(
    go.Bar(
        name='Precision',
        x=categories,
        y=precision,
        marker_color='royalblue',
        width=0.2,
        offset=-0.2,
        hovertemplate="Category: %{x}<br>Precision: %{y:.2f}<extra></extra>"
    ),
    row=2, col=1
)

fig.add_trace(
    go.Bar(
        name='Recall',
        x=categories,
        y=recall,
        marker_color='lightgreen',
        width=0.2,
        offset=0,
        hovertemplate="Category: %{x}<br>Recall: %{y:.2f}<extra></extra>"
    ),
    row=2, col=1
)

fig.add_trace(
    go.Bar(
        name='F1-Score',
        x=categories,
        y=f1,
        marker_color='salmon',
        width=0.2,
        offset=0.2,
        hovertemplate="Category: %{x}<br>F1-Score: %{y:.2f}<extra></extra>"
    ),
    row=2, col=1
)

fig.update_layout(
    height=1200,
    width=1200,
    showlegend=True,
    title={
        'text': "Model Evaluation Dashboard",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24)
    },
    barmode='group'
)

fig.update_xaxes(
    title_text="Predicted Label",
    row=1,
    col=1,
    tickangle=45,
    tickfont=dict(size=10)
)
fig.update_yaxes(
    title_text="True Label",
    row=1,
    col=1,
    tickfont=dict(size=10)
)
fig.update_xaxes(
    title_text="Categories",
    row=2,
    col=1,
    tickangle=45,
    tickfont=dict(size=10)
)
fig.update_yaxes(
    title_text="Score",
    row=2,
    col=1
)

fig.show()

fig.write_html("model_evaluation.html")

In [ ]:
import torch
import modelbit
from transformers import AutoModelForImageClassification, AutoProcessor
from torchvision import transforms
from PIL import Image
import io
import base64

def FoodSnap_Prediction_v50(image_input):
    categories = [
        'adhirasam', 'aloo_gobi', 'aloo_matar', 'aloo_methi', 'aloo_shimla_mirch',
        'aloo_tikki', 'anarsa', 'ariselu', 'bandar_laddu', 'basundi', 'bhatura',
        'bhindi_masala', 'biryani', 'boondi', 'butter_chicken', 'chak_hao_kheer',
        'cham_cham', 'chana_masala', 'chapati', 'chhena_kheeri', 'chicken_razala',
        'chicken_tikka', 'chicken_tikka_masala', 'chikki', 'daal_baati_churma',
        'daal_puri', 'dal_makhani', 'dal_tadka', 'dharwad_pedha', 'doodhpak',
        'double_ka_meetha', 'dum_aloo', 'gajar_ka_halwa', 'gavvalu', 'ghevar',
        'gulab_jamun', 'imarti', 'jalebi', 'kachori', 'kadai_paneer',
        'kadhi_pakoda', 'kajjikaya', 'kakinada_khaja', 'kalakand', 'karela_bharta',
        'kofta', 'kuzhi_paniyaram', 'lassi', 'ledikeni', 'litti_chokha',
        'lyangcha', 'maach_jhol', 'makki_di_roti_sarson_da_saag', 'malapua',
        'misi_roti', 'misti_doi', 'modak', 'mysore_pak', 'naan', 'navrattan_korma',
        'palak_paneer', 'paneer_butter_masala', 'phirni', 'pithe', 'poha',
        'poornalu', 'pootharekulu', 'qubani_ka_meetha', 'rabri', 'ras_malai',
        'rasgulla', 'sandesh', 'shankarpali', 'sheer_korma', 'sheera',
        'shrikhand', 'sohan_halwa', 'sohan_papdi', 'sutar_feni', 'unni_appam'
    ]
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    model = modelbit.get_model("FoodSnap_V50")
    model.eval()

    if isinstance(image_input, str):
        image_data = base64.b64decode(image_input)
        image = Image.open(io.BytesIO(image_data)).convert("RGB")
    elif isinstance(image_input, bytes):
        image = Image.open(io.BytesIO(image_input)).convert("RGB")
    else:
        raise ValueError("The input should be a base64 string (str) or raw image bytes (bytes).")

    image_tensor = transform(image).unsqueeze(0)

    with torch.no_grad():
        outputs = model(image_tensor).logits

    _, predicted_class = torch.max(outputs, 1)
    prediction_prob = torch.nn.functional.softmax(outputs, dim=1)[0, predicted_class].item()
    predicted_label = categories[predicted_class.item()]

    return predicted_label, prediction_prob

modelbit.deploy(FoodSnap_Prediction_v50, python_packages=["transformers==4.44.2"])